In [1]:
import torch 
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
transform_train= transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
])
transform_test = transforms.Compose([
    transforms.ToTensor()
])

In [6]:
import torchvision 
trainset = torchvision.datasets.CIFAR100(root='/.data', train=True, download=True, transform=transform_train)
trainloader  = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

100%|██████████| 169M/169M [00:03<00:00, 43.2MB/s] 


In [7]:
testset = torchvision.datasets.CIFAR100(root='/.data', train=False, download=True, transform=transform_test)
testloader  = DataLoader(trainset, batch_size=64, shuffle=False, num_workers=2)

In [8]:
len(trainset), len(testset)

(50000, 10000)

In [9]:
class_names  = trainset.classes
class_names[:10]

['apple',
 'aquarium_fish',
 'baby',
 'bear',
 'beaver',
 'bed',
 'bee',
 'beetle',
 'bicycle',
 'bottle']

In [10]:
model = torchvision.models.resnet18(weights=False).to(device)
model.fc  = nn.Linear(model.fc.in_features, 100)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
import torch.optim as optim
loss_fn  = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=0.01)

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.ep(y_pred, y_true).sum().item()
    acc = correct/len(y_pred)
    return acc

In [18]:
#create a training loop
def train_step(model:torch.nn.Module,
               data_loader:torch.utils.data.dataloader,
               optimizer:torch.optim.Optimizer,
               loss_fn:torch.nn.Module,
               accuracy_fn,
               device: torch.device= device):
    train_acc, train_loss = 0, 0 
    model.train()
    for batch, (X, Y) in enumerate(data_loader):
        X, Y = X.to(device), Y.to(device)
        y_pred  = model(X)
        loss = loss_fn(y_pred, Y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=Y, y_pred=y_pred.argmex(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_acc /= len(data_loader)
        train_acc /= len(data_loader)
        print(f"train loss: {train_loss:.5f}  | train acc: {train_acc:.5f}% | lr: {scheduler.get_last_lr()[0]:.5f}")